In [ ]:
import cv2
import glob
import os
from skimage.feature import hog
# from sklearn.svm import LinearSVC
from sklearn.svm import LinearSVC
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA

# Load the dataset of images
# Each image is a grayscale image of size 64x128
dataset_path = '0/'
image_paths = glob.glob(os.path.join(dataset_path, '*.JPG'))

# Define the HOG parameters
orientations = 9
pixels_per_cell = (8, 8)
cells_per_block = (3, 3)
block_norm = 'L2-Hys'

# Extract the HOG features from each image
hog_features = []
labels = []
for image_path in image_paths:
    image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    hog_feature = hog(image, orientations=orientations,
                      pixels_per_cell=pixels_per_cell,
                      cells_per_block=cells_per_block,
                      block_norm=block_norm)
    hog_features.append(hog_feature)
    label = os.path.basename(image_path).split('.')[0][0]
    labels.append(int(label))

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(hog_features, labels, test_size=0.2, random_state=42)

# Apply PCA to the data 
pca = PCA(n_components=10)
pca.fit(X_train)
X_train_pca = pca.transform(X_train)
X_test_pca = pca.transform(X_test)

# Train the SVM classifier on the PCA-transformed data
svm = LinearSVC()
svm.fit(X_train_pca, y_train)

# Evaluate the classifier on the PCA-transformed testing data
y_pred = svm.predict(X_test_pca)
print('y_test',y_test)
print('y_pred',y_pred)
accuracy = accuracy_score(y_test, y_pred)

print('Accuracy:', accuracy)